# <font color=blue><div align="center">Seats Allocation within an Aircraft</div></font>



## Modules

In [9]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Préprocessing des données

Pour la suite de notre étude, on considère le modèle d'avion **Airbus A320**, qui dispose de **174** sièges. (La classe businees n'est pas prise en compte)


<img src="modele_avion.jpg" alt="Modèle de l'avion" width="400" height="600">

In [5]:
number_of_seats = 174

In [4]:
# Load the Excel file into a pandas DataFrame

df_21Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=0 , skipfooter=2)
df_22Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=1 , skipfooter=2)
df_23Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=2 , skipfooter=2)
df_24Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=3 , skipfooter=2)
df_30Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=4 , skipfooter=2)
df_05Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=5 , skipfooter=2)
df_07Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=6 , skipfooter=2)


In [6]:
# Displaying the data

df_07Nov.head()

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,1.0,NaN,NaN,00:55:00
1,2,NaN,1.0,NaN,00:00:00
2,3,1.0,NaN,NaN,01:15:00
3,4,1.0,1.0,NaN,03:15:00
4,5,1.0,NaN,NaN,00:00:00


In [7]:
# On choisit la df sur laquelle on va travailler

df = df_21Oct

In [8]:
# Initialize the model
m = Model("airplane_seating")

# Assuming 'seats' is a list of tuples (i, j) representing seat positions
# And 'passengers' is a list of all passengers indexed by k

# Decision variables
x = m.addVars(seats, passengers, vtype=GRB.BINARY, name="x")

# Constraints
# Each seat can only be assigned to one passenger
for seat in seats:
    m.addConstr(quicksum(x[seat, k] for k in passengers) <= 1, "SeatLimit_%s" % str(seat))

# Each passenger must have one seat
for k in passengers:
    m.addConstr(quicksum(x[i, j, k] for i, j in seats) == 1, "PassengerSeat_%s" % k)

# Additional constraints from the handwritten notes and PDF will be added here

# Objective function
# Minimize the sum of weighted distances for all groups
objective = quicksum(weight * x[seat, k] for seat in seats for k in passengers)
m.setObjective(objective, GRB.MINIMIZE)

# Optimize the model
m.optimize()

# Post-processing of results
# Extract the seating arrangement from the model
seating_arrangement = {k: seat for seat in seats for k in passengers if x[seat, k].X > 0.5}


Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-04


NameError: name 'seats' is not defined

In [ ]:
# Assuming the setup from your initial code snippet continues...
# and the data includes all passengers' details and their groups

# Define model
m = Model("airplane_seating")

# Define decision variables based on your passenger data, seats available, and any other parameters

# Add constraints
# Each passenger must be seated and seats constraints
# Group seating constraints
# Special needs constraints (handicap accessible seating)
# Weight distribution constraints
# Any other constraints from the notes

# Set objective function
# It can be a combination of minimizing distance, travel time, etc.

# Optimize model
m.optimize()

# Process results
# Extract seating arrangement
# Apply any post-optimization analysis or processing

# End of Model
